In [61]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import boto3
import os
import time
import io
import pyathena as cz
from pyathena.pandas_cursor import PandasCursor
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

In [2]:
# s3 = boto3.resource('s3')
athena_conection = "athena.us-east-1.amazonaws.com"
region_name = ('us-east-1')
db = "ml_platform_events"

In [3]:
athena_client   = boto3.client('athena', region_name='us-east-1', aws_access_key_id=os.environ['ACCESS_KEY'],
                            aws_secret_access_key=os.environ['ACCESS_SECRET_KEY'])

s3_resource   = boto3.resource('s3', region_name='us-east-1', aws_access_key_id=os.environ['ACCESS_KEY'],
                            aws_secret_access_key=os.environ['ACCESS_SECRET_KEY'])

In [4]:
def athena_query_to_dataframe(db, query):
    
    listOfStatus = ['SUCCEEDED', 'FAILED', 'CANCELLED']
    listOfInitialStatus = ['RUNNING', 'QUEUED']
    
    print('Starting Query Execution:')
       
    response = athena_client.start_query_execution(
        QueryString = query,
        QueryExecutionContext = {
            'Database': db
        },
        ResultConfiguration = {
            'OutputLocation': 's3://personal-us-east-1/query-result/',
        }
    )

    queryExecutionId = response['QueryExecutionId']

    status = athena_client.get_query_execution(QueryExecutionId = queryExecutionId)['QueryExecution']['Status']['State']

    while status in listOfInitialStatus:
        status = athena_client.get_query_execution(QueryExecutionId = queryExecutionId)['QueryExecution']['Status']['State']
        if status in listOfStatus:
            if status == 'SUCCEEDED':
                print('Query Succeeded!')
                paginator = athena_client.get_paginator('get_query_results')
                query_results = paginator.paginate(
                    QueryExecutionId = queryExecutionId,
                    PaginationConfig = {'PageSize': 1000}
                )
            elif status == 'FAILED':
                print('Query Failed!')
            elif status == 'CANCELLED':
                print('Query Cancelled!')
            break
    
    results = []
    rows = []
    
    print('Processing Response')
    
    for page in query_results:
        for row in page['ResultSet']['Rows']:
            rows.append(row['Data'])

    columns = rows[0]
    rows = rows[1:]

    columns_list = []
    for column in columns:
        columns_list.append(column['VarCharValue'])
        
    print('Creating Dataframe')

    dataframe = pd.DataFrame(columns = columns_list)

    for row in rows:
        df_row = []
        for data in row:
            df_row.append(data['VarCharValue'])
        dataframe.loc[len(dataframe)] = df_row
    
        
    print('Done!')
    
    return(dataframe)

In [5]:
df = athena_query_to_dataframe(db, "select * from ml_platform_events.event_punkapi")

Starting Query Execution:


ClientError: An error occurred (UnrecognizedClientException) when calling the StartQueryExecution operation: The security token included in the request is invalid.

In [130]:
df.head(1000)

,name,abv,ibu,target_fg,target_og,ebc,srm,ph
0,Galaxy,6.7,67.0,1064,1013,19.0,9.50,4.4
1,10 Heads High,7.8,70.0,1015,1074,90.0,45.00,4.4
2,Lumberjack Stout,15.2,20.0,1020,1110,158.0,79.00,4.4
3,Neon Overlord,7.6,70.0,1012,1067,20.0,10.00,4.4
4,Ship Wreck,13.8,60.0,1015,1088,25.0,13.00,4.4
5,Crew Brew,8.0,100.0,1011,1072,28.0,14.20,5.2
6,The End Of History,55.0,NaN,1000,1112,NaN,NaN,4.4
7,Movember,4.5,50.0,1012,1047,140.0,70.00,5.2
8,Sunmaid Stout,10.2,50.0,1102,1026,197.0,100.00,4.4
9,AB:07,12.5,30.0,1020,1106,84.0,42.00,5.6


In [20]:
df = pd.read_csv("beers.csv")
# df = df.drop(['index', 'id'], axis=1)
# df.round({'abv':2})

In [8]:
df.dropna().to_csv("beers.csv")

In [114]:
## convert

df["ibu"] = pd.to_numeric(df["ibu"], downcast="float")
df["abv"] = pd.to_numeric(df["abv"], downcast="float")
df["target_fg"] = pd.to_numeric(df["target_fg"], downcast="integer")
df["target_og"] = pd.to_numeric(df["target_og"], downcast="integer")
df["ebc"] = pd.to_numeric(df["ebc"], downcast="integer")
df["srm"] = pd.to_numeric(df["srm"], downcast="integer")
df["ph"] = pd.to_numeric(df["ph"], downcast="float")
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]


df.dtypes
df.head()

,name,abv,ibu,target_fg,target_og,ebc,srm,ph
0,Galaxy,6.7,67.0,1064,1013,19,9.5,4.4
1,10 Heads High,7.8,70.0,1015,1074,90,45.0,4.4
2,Lumberjack Stout,15.2,20.0,1020,1110,158,79.0,4.4
3,Neon Overlord,7.6,70.0,1012,1067,20,10.0,4.4
4,Ship Wreck,13.8,60.0,1015,1088,25,13.0,4.4


In [115]:
df.dropna(inplace=True)

In [116]:
## seperar train e test
features = [df["abv"],df['target_fg'],	df['target_og'],df['ebc'], df['srm'],df['ph']]
X = df[features] ## features
y = df.label ## target

KeyError: '[ (6.699999809265137, 7.800000190734863, 15.199999809265137, 7.599999904632568, 13.800000190734863, 8.0, 4.5, 10.199999809265137, 12.5, 10.0, 8.5, 13.5, 8.5, 5.199999809265137)\n (1064, 1015, 1020, 1012, 1015, 1011, 1012, 1102, 1020, 1012, 1007, 1015, 1003, 1014)\n (1013, 1074, 1110, 1067, 1088, 1072, 1047, 1026, 1106, 1098, 1073, 1112, 1068, 1052)\n (19, 90, 158, 20, 25, 28, 140, 197, 84, 30, 15, 300, 15, 200)\n (9.5, 45.0, 79.0, 10.0, 13.0, 14.199999809265135, 70.0, 100.0, 42.0, 15.25, 8.0, 152.0, 7.5, 100.0)\n (4.400000095367432, 4.400000095367432, 4.400000095367432, 4.400000095367432, 4.400000095367432, 5.199999809265137, 5.199999809265137, 4.400000095367432, 5.599999904632568, 4.400000095367432, 4.199999809265137, 4.400000095367432, 4.400000095367432, 5.199999809265137)] not in index'

In [103]:
X.head()


0    67.0
1    70.0
2    20.0
3    70.0
4    60.0
Name: ibu, dtype: float32

In [104]:
len(y)

14

In [105]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [106]:
X_train = X_train.values.reshape(-1, 1)

In [107]:
X_test = X_test.values.reshape(-1, 1)

In [108]:
logisticRegr = LogisticRegression()

In [109]:
logisticRegr.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [110]:
logisticRegr.predict(X_test)

array(['Crew Brew', 'Crew Brew', 'Crew Brew', 'Crew Brew', 'Crew Brew'], dtype=object)